![An interactive LADAL notebook](https://slcladal.github.io/images/uq1.jpg)

# Semantic Vector Space Models in R

This tutorial is the interactive Jupyter notebook accompanying the [*Language Technology and Data Analysis Laboratory* (LADAL) tutorial *Semantic Vector Space Models in R*](https://ladal.edu.au/svm.html). The tutorial provides more details and background information while this interactive notebook focuses strictly on practical aspects.


**Preparation and session set up**

We set up our session by activating the packages we need for this tutorial. 


In [ ]:
# activate packages
library(coop)
library(dplyr)
library(tm)
library(cluster)


Once you have initiated the session by executing the code shown above, you are good to go.

If you are using this notebook on your own computer and you have not already installed the R packages listed above, you need to install them. You can install them by replacing the `library` command with `install.packages` and putting the name of the package into quotation marks like this: `install.packages("dplyr")`. Then, you simply run this command and R will install the package you specified.


***

## Using your own data

While the tutorial uses data from the LADAL website, you can also use your own data. You can see below what you need to do to upload and use your own data.

The code chunk below allows you to upload two files from your own computer. To be able to load your own data, you need to click on the folder symbol to the left of the screen:

![Binder Folder Symbol](https://slcladal.github.io/images/binderfolder.JPG)


Then on the upload symbol.

![Binder Upload Symbol](https://slcladal.github.io/images/binderupload.JPG)

Next, upload the files you want to analyze and then the respective files names in the file argument of the scan function. When you then execute the code (like to code chunk below, you will upload your own data.


In [ ]:
mytable1 <- openxlsx::read.xlsx("testdata1.xlsx", sheet = 1)
# inspect
mytable1


**Keep in mind though that you need to adapt the names of the files in the code chunks below so that the code below work on your own data!**

***

# Example: Similarity among adjective amplifiers

Adjective amplifiers are degree adverbs such as *very*, *really*, or *awfully* as those shown in 1. to 5.

1. The *very*~amplifier~ *nice*~adjective~ man.
2. A *truely*~amplifier~ *remarkable*~adjective~ woman. 
2. He was *really*~amplifier~ *hesitant*~adjective~.
4. The child was *awefully*~amplifier~ *loud*~adjective~.
5. The festival was *so*~amplifier~ *amazing*~adjective~!

We start by loading an already existing data set containing amplifer-adjective bigrams.


In [ ]:
# load data
vsmdata <- read.delim("https://slcladal.github.io/data/vsmdata.txt",
                      sep = "\t", header = T)
# inspect
head(vsmdata)


For this tutorial, we remove adjectives that were not amplified (as well as adjectives modified by *much* or *many*) and collapse all adjectives that occur less than 10 times into a bin category (*other*).



In [ ]:
# simplify data
vsmdata_simp <- vsmdata %>%
  # remove non-amplifier adjectives
  dplyr::filter(Amplifier != 0,
         Adjective != "many",
         Adjective != "much") %>%
  # collapse infrequent adjectives
  dplyr::group_by(Adjective) %>%
  dplyr::mutate(AdjFreq = dplyr::n()) %>%
  dplyr::ungroup() %>%
  dplyr::mutate(Adjective = ifelse(AdjFreq > 10, Adjective, "other")) %>%
  dplyr::filter(Adjective != "other") %>%
  dplyr::select(-AdjFreq)
# inspect
head(vsmdata_simp)


In a next step, we create a text-document matrix or tdm.



In [ ]:
# tabulate data (create term-document matrix)
tdm <- ftable(vsmdata_simp$Adjective, vsmdata_simp$Amplifier)
# extract amplifiers and adjectives 
amplifiers <- as.vector(unlist(attr(tdm, "col.vars")[1]))
adjectives <- as.vector(unlist(attr(tdm, "row.vars")[1]))
# attach row and column names to tdm
rownames(tdm) <- adjectives
colnames(tdm) <- amplifiers
# inspect data
tdm[1:5, 1:5]


Now that we have a term document matrix, we want to remove adjectives that were never amplified. 



In [ ]:
# convert frequencies greater than 1 into 1
tdm <- t(apply(tdm, 1, function(x){ifelse(x > 1, 1, x)}))
# remove adjectives that we never amplified
tdm <- tdm[which(rowSums(tdm) > 1),]
# inspect data
tdm[1:5, 1:5]


In a next step, we extract the expected values of the co-occurrences if the amplifiers were distributed homogeneously and calculate the *Pointwise Mutual Information* (PMI) score and use that to then calculate the *Positive Pointwise Mutual Information* (PPMI) scores. 



In [ ]:
# compute expected values
tdm.exp <- chisq.test(tdm)$expected
# calculate PMI and PPMI
PMI <- log2(tdm/tdm.exp)
PPMI <- ifelse(PMI < 0, 0, PMI)
# calculate cosine similarity
cosinesimilarity <- cosine(PPMI)
# inspect cosine values
cosinesimilarity[1:5, 1:5]


As we have now obtained a similarity measure, we can go ahead and perform a cluster analysis on these similarity values. However, as we have to extract the maximum values in the similarity matrix that is not 1 as we will use this to create a distance matrix.



In [ ]:
# find max value that is not 1
cosinesimilarity.test <- apply(cosinesimilarity, 1, function(x){
  x <- ifelse(x == 1, 0, x) } )
maxval <- max(cosinesimilarity.test)
# create distance matrix
amplifier.dist <- 1 - (cosinesimilarity/maxval)
clustd <- as.dist(amplifier.dist)


In a next step, we want to determine the optimal number of clusters. 



In [ ]:
# find optimal number of clusters
asw <- as.vector(unlist(sapply(2:nrow(tdm)-1, function(x) pam(clustd, k = x)$silinfo$avg.width)))
# determine the optimal number of clusters (max width is optimal)
optclust <- which(asw == max(asw)) # optimal number of clusters
# inspect clustering with optimal number of clusters
amplifier.clusters <- pam(clustd, optclust)
# inspect cluster solution
amplifier.clusters$clustering


In a next step, we visualize the results of the semantic vector space model as a dendrogram.



In [ ]:
# create cluster object
cd <- hclust(clustd, method="ward.D")    
# plot cluster object
plot(cd, main = "", sub = "", yaxt = "n", ylab = "", xlab = "", cex = .8)
# add colored rectangles around clusters
rect.hclust(cd, k = 6, border = "gray60")


***

[Back to LADAL](https://ladal.edu.au/llr.html)

***
